In [51]:
know = {
    "intents" : {
        "saudacao" : [
            "Oi", "Olá"
        ],
        "despedida" : [
            "tchau", "Até mais!"
        ],
        "saldo" : [
            "Qual é meu saldo?", "Quanto tenho na conta?", "Saldo da conta corrente", "Quantas pilas eu tenho?", "Qual é o saldo da minha conta?"
        ],
        "extrato" : [
            "Extrato da conta corrente"
        ],
        "recarregar-celular" : [
            "Quero recarregar meu celular!"
        ],
        "transferir" : [
            "Transferir dinheiro"
        ]
    }
}
print(know);

{'intents': {'extrato': ['Extrato da conta corrente'], 'despedida': ['tchau', 'Até mais!'], 'saudacao': ['Oi', 'Olá'], 'recarregar-celular': ['Quero recarregar meu celular!'], 'saldo': ['Qual é meu saldo?', 'Quanto tenho na conta?', 'Saldo da conta corrente', 'Quantas pilas eu tenho?', 'Qual é o saldo da minha conta?'], 'transferir': ['Transferir dinheiro']}}


In [20]:
dialog_welcome = {
    "condition": "#saudacao",
    "output": {
        "text": "Olá, eu sou o David!"
    }
}

dialog_nodes = [dialog_welcome,{
    "condition": "#despedida",
    "output": {
        "text": "Muito obrigado pela atenção! Precisando estou por aqui!"
    } 
}, {
    "condition": "#saldo",
    "output": {
        "text": "Seu saldo é R$ #.###,##!"
    } 
},{
    "condition": "#extrato",
    "output": {
        "text": "Segue seu extrato!!!"
    } 
},{
    "condition": "#recarregar-celular",
    "output": {
        "text": "Celular recarregado!"
    } 
},{
    "condition": "#transferir",
    "output": {
        "text": "Dinheiro transferido!"
    } 
}]

dialog_anythinelse = { 
    "condition" : "true",
    "output": {
        "text": "Desculpe, não entendi! Pode falar de outra forma?"
    }
}

dialog_nodes.append(dialog_anythinelse)

print(dialog_nodes);

[{'condition': '#saudacao', 'output': {'text': 'Olá, eu sou o David!'}}, {'condition': '#despedida', 'output': {'text': 'Muito obrigado pela atenção! Precisando estou por aqui!'}}, {'condition': '#saldo', 'output': {'text': 'Seu saldo é R$ #.###,##!'}}, {'condition': '#extrato', 'output': {'text': 'Segue seu extrato!!!'}}, {'condition': '#recarregar-celular', 'output': {'text': 'Celular recarregado!'}}, {'condition': '#transferir', 'output': {'text': 'Dinheiro transferido!'}}, {'condition': 'true', 'output': {'text': 'Desculpe, não entendi! Pode falar de outra forma?'}}]


In [44]:
stopwords = set(line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8'))

In [42]:
import re

NON_CONTENT = r"[^\w\d\s]"
def tokenize(sentence):
    #remove non content
    sentence = re.sub(NON_CONTENT, "", sentence)
    #lower
    sentence = sentence.lower();
    
    #split
    tokens = sentence.split(" ");
        
    for sw in stopwords:
        try:
            tokens.remove(sw); 
        except ValueError:
            pass
    
    return tokens;

print(tokenize("Qual é o meu saldo?"))

['saldo']


In [52]:
intent_model = {}

for intent, samples in know["intents"].items():
    intent_model[intent] = {
        "total" : 0,
        "tokens" : {}
    }
    for sample in samples: 
        for t in tokenize(sample):
            intent_model[intent]["total"] += 1;
            if t in intent_model[intent]["tokens"]:
                intent_model[intent]["tokens"][t] += 1;
            else:
                intent_model[intent]["tokens"][t] = 1;
    
print("intent_model:", intent_model);    

intent_model: {'extrato': {'tokens': {'extrato': 1, 'conta': 1}, 'total': 2}, 'despedida': {'tokens': {'tchau': 1}, 'total': 1}, 'saudacao': {'tokens': {'olá': 1, 'oi': 1}, 'total': 2}, 'recarregar-celular': {'tokens': {'recarregar': 1, 'celular': 1}, 'total': 2}, 'saldo': {'tokens': {'saldo': 3, 'conta': 3, 'pilas': 1}, 'total': 7}, 'transferir': {'tokens': {'dinheiro': 1, 'transferir': 1}, 'total': 2}}


In [47]:
def classify(input):
    tokens = tokenize(input);
    #print ("tokens", tokens);
    intents = {}
    for intent, imeta in intent_model.items(): 
        brutal_score = 0
        itokens = imeta["tokens"]
        for t in tokens:
            if t in itokens:
                brutal_score += itokens[t]
        intents[intent] = brutal_score / imeta["total"]       
    
    intents = [{
        "intent": intent,
        "confidence": intents[intent]
    } for intent in sorted(intents, key=intents.__getitem__, reverse = True)]
    
    intents = list(filter(lambda i: i["confidence"] > 0, intents))
    
    return intents[:10];

print(classify("Qual meu saldo?"))

[{'confidence': 0.4, 'intent': 'saldo'}]


In [6]:
def nlp(input):
    return [];

In [7]:
def evalCondition(condition, context, intent, entities):
    return condition == '#' + intent;

In [8]:
def dialog(input, context, intents, entities):
    if input == "":
        return dialog_welcome;
    
    if len(intents) > 0:
        intent = intents[0]["intent"];
        for dialog_node in dialog_nodes:
            if evalCondition(dialog_node["condition"], context, intent, entities):
                return dialog_node;
    
    return dialog_anythinelse;

In [9]:
def respond(input, context = {}):
    intents = classify(input)
    entities = nlp(input)
    #print("intents", intents)
    dialog_node = dialog(input, context, intents, entities)
    return {
        "context": context,
        "intents": intents,
        "entities": entities,
        "output": dialog_node["output"]
    }

In [54]:
def test_respond(input):
    resp = respond(input);
    #print("\n\nresp", resp);
    print("\ninput: ", input, " >> ", resp["output"]["text"]);

test_respond("");
test_respond("Oi");
test_respond("Qual meu saldo?");
test_respond("Quanto tenho na conta?");
test_respond("Extrato da conta");
test_respond("link pra receber");
test_respond("tchau");


input:    >>  Olá, eu sou o David!

input:  Oi  >>  Olá, eu sou o David!

input:  Qual meu saldo?  >>  Seu saldo é R$ #.###,##!

input:  Quanto tenho na conta?  >>  Segue seu extrato!!!

input:  Extrato da conta  >>  Segue seu extrato!!!

input:  link pra receber  >>  Desculpe, não entendi! Pode falar de outra forma?

input:  tchau  >>  Muito obrigado pela atenção! Precisando estou por aqui!
